In [248]:
import pandas as pd
import numpy as np
from getpass import getpass
import os
from dotenv import load_dotenv
import requests
import json
from sklearn.preprocessing import OneHotEncoder
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import metrics

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

In [4]:
!pwd


/Users/leonplaza/Desktop/ironhack/lab/technical_challenge_DA


In [2]:
token = os.getenv("token")
load_dotenv()

url = "https://www.high-mobility.com/?mtm_campaign=S-EU-EN-Kw-CarAPI"

headers = {"accept": "application/json",
          "authorization": token }

response = requests.get(url)
response

In [205]:
def specials_clean(x):
    if x == "AC sun":
        return "AC Sun"
    elif x == "sun ac":
        return "AC Sun"
    
    elif x == "AC rain":
        return "AC Rain"
    elif x == "ac rain":
        return "AC Rain"
    else:
        return x
    
    
    
def ac_on(x):
    if x == "AC sun":
        return 1
    elif x == "sun ac":
        return 1
    elif x == "AC rain":
        return 1
    elif x == "ac rain":
        return 1
    elif x == "AC Rain":
        return 1
    elif x == "AC Sun":
        return 1
    elif x == "AC snow":
        return 1
    elif x == "AC":
        return 1
    elif x == "ac":
        return 1
    else:
        return 0
    


def gas_type(x):
    if x == "E10":
        return 0
    elif x == "SP98":
        return 1
    else:
        return x
    


In [210]:
df = pd.read_csv("data/measurements.csv")
data = df
data.rename({"distance":"km","consume":"L/100km","speed":"km/h"}, axis=1)

data.gas_type = data.gas_type.apply(lambda x: gas_type(x))
data.specials = data.specials.apply(lambda x: specials_clean(x))

#oneonehotencoder = OneHotEncoder()
#data[list(set(data.specials))] = oneonehotencoder.fit_transform(data["specials"].values.reshape(-1, 1)).toarray()

data.temp_inside = data.temp_inside.astype(str).apply(lambda x: float(x.replace(",",".")))
data.temp_outside = data.temp_outside.astype(str).apply(lambda x: float(x.replace(",",".")))
data.consume = data.consume.astype(str).apply(lambda x: float(x.replace(",",".")))
data.distance = data.distance.astype(str).apply(lambda x: float(x.replace(",",".")))

data["temp_diff"] = data.temp_inside - data.temp_outside
data["time"] = data.distance / data.speed
data["liters"] = data.distance / (100/(data.consume)) 
data["AC ON"] = data.specials.apply(lambda x: ac_on(x))
data.drop(["refill gas","refill liters"], axis=1, inplace=True)

In [211]:
data

,distance,consume,speed,temp_inside,temp_outside,specials,gas_type,AC,rain,sun,temp_diff,time,liters,AC ON
0,28.0,5.0,26,21.5,12.0,NaN,0,0,0,0,9.5,1.076923,1.4000,0
1,12.0,4.2,30,21.5,13.0,NaN,0,0,0,0,8.5,0.400000,0.5040,0
2,11.2,5.5,38,21.5,15.0,NaN,0,0,0,0,6.5,0.294737,0.6160,0
3,12.9,3.9,36,21.5,14.0,NaN,0,0,0,0,7.5,0.358333,0.5031,0
4,18.5,4.5,46,21.5,15.0,NaN,0,0,0,0,6.5,0.402174,0.8325,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,16.0,3.7,39,24.5,18.0,NaN,1,0,0,0,6.5,0.410256,0.5920,0
384,16.1,4.3,38,25.0,31.0,AC,1,1,0,0,-6.0,0.423684,0.6923,1
385,16.0,3.8,45,25.0,19.0,NaN,1,0,0,0,6.0,0.355556,0.6080,0
386,15.4,4.6,42,25.0,31.0,AC,1,1,0,0,-6.0,0.366667,0.7084,1


In [213]:
data.select_dtypes(exclude="object").groupby("AC ON").mean()

,distance,consume,speed,temp_inside,temp_outside,gas_type,AC,rain,sun,temp_diff,time,liters
AC ON,,,,,,,,,,,,
0,19.465084,4.904749,41.969274,21.833815,10.986034,0.575419,0.002793,0.103352,0.075419,10.880058,0.438889,0.925378
1,21.893333,5.003333,41.433333,23.033333,15.800000,0.733333,0.966667,0.366667,0.166667,7.233333,0.505597,1.053630


In [196]:
#sns.heatmap(data.corr())
data.groupby("specials").count()

,distance,consume,speed,temp_inside,temp_outside,gas_type,AC,rain,sun,AC Rain,snow,NaN,AC snow,ac,AC Sun,temp_diff,time,liters
specials,,,,,,,,,,,,,,,,,,
AC,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
AC Rain,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
AC Sun,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
AC snow,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
ac,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
half rain half sun,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
rain,32,32,32,31,32,32,32,32,32,32,32,32,32,32,32,31,32,32
snow,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
sun,27,27,27,26,27,27,27,27,27,27,27,27,27,27,27,26,27,27


In [246]:
data.loc[:,data.columns != "specials"].dropna()
X

,distance,speed,temp_inside,temp_outside,gas_type,AC,rain,sun,temp_diff,time,liters,AC ON
0,28.0,26,21.5,12.0,0,0,0,0,9.5,1.076923,1.4000,0
1,12.0,30,21.5,13.0,0,0,0,0,8.5,0.400000,0.5040,0
2,11.2,38,21.5,15.0,0,0,0,0,6.5,0.294737,0.6160,0
3,12.9,36,21.5,14.0,0,0,0,0,7.5,0.358333,0.5031,0
4,18.5,46,21.5,15.0,0,0,0,0,6.5,0.402174,0.8325,0
...,...,...,...,...,...,...,...,...,...,...,...,...
383,16.0,39,24.5,18.0,1,0,0,0,6.5,0.410256,0.5920,0
384,16.1,38,25.0,31.0,1,1,0,0,-6.0,0.423684,0.6923,1
385,16.0,45,25.0,19.0,1,0,0,0,6.0,0.355556,0.6080,0
386,15.4,42,25.0,31.0,1,1,0,0,-6.0,0.366667,0.7084,1


In [245]:
data1 = data.loc[:,data.columns != "specials"].dropna()
y = data1.consume
X = data1.loc[:,data1.columns != "consume"]



X1_train, X1_test, y1_train, y1_test = train_test_split(X, y, test_size=0.35)

In [249]:
models = {
    "Linear Regression": LinearRegression(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Gradient Boosting Regressor": GradientBoostingRegressor(),
    "SGD Regressor": SGDRegressor(),
    "SVR": SVR(),
    "DecisionTreeRegressor":DecisionTreeRegressor(max_depth=4)
}

for model in models.values():
#    print(f"Training: {model}")
    model.fit(X1_train, y1_train)
    
metric = []
index_models = []
for name, model in models.items():
    y1_pred = model.predict(X1_test)
    mae = metrics.mean_absolute_error(y1_test, y1_pred)    
    mse = metrics.mean_squared_error(y1_test, y1_pred)
    rmse = np.sqrt(metrics.mean_squared_error(y1_test, y1_pred))
    r2 = metrics.r2_score(y1_test, y1_pred)
    metric.append([mae,mse,rmse,r2])
    index_models.append(name)
    
results1 = pd.DataFrame(metric,index=index_models, columns=["MAE","MSE","RMSE","R2"])
results1["TradeOff"] = results1.RMSE / results1.R2
results1

,MAE,MSE,RMSE,R2,TradeOff
Linear Regression,4.643172e-01,1.207808e+00,1.099003e+00,5.477402e-03,2.006432e+02
K-Neighbors Regressor,4.878788e-01,4.928121e-01,7.020058e-01,5.942131e-01,1.181404e+00
Gradient Boosting Regressor,2.657820e-01,2.634955e-01,5.133181e-01,7.830349e-01,6.555494e-01
SGD Regressor,6.558197e+11,4.509575e+23,6.715337e+11,-3.713233e+23,-1.808488e-12
SVR,5.218730e-01,7.964262e-01,8.924271e-01,3.442140e-01,2.592652e+00
DecisionTreeRegressor,5.193929e-01,6.000482e-01,7.746278e-01,5.059138e-01,1.531146e+00
